# Training Distil-Bert (Finetuning For Disease Dataset To Get Disease Codes)

In [14]:
import torch, os
import pandas as pd
from transformers import pipeline, BertForSequenceClassification, BertTokenizerFast
from torch.utils.data import Dataset

In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cpu'

In [28]:
ROOT_DIR = '' 


df_org= pd.read_csv(ROOT_DIR + "diseaseOne.csv")

df_org = df_org.sample(frac=1.0, random_state=42)

df_org.head(10000)

,Code,Disease
328,70482,Ct orbit/ear/fossa w/ow/dye
689,78700,Kidney imaging morphol
413,72197,Mri pelvis w/o w/dye
720,93306,Tte w/doppler complete
244,97546,Work hardening add-on
...,...,...
466,73702,Ct lwr extremity w/ow/dye
121,0335U,Rare ds whl gen seq feta
1044,90657,IIV3 vaccine splt 0.25 ml im
1095,Q2034,Agriflu vaccine


In [5]:
labels = df_org['Code'].unique().tolist()
labels = [s.strip() for s in labels ]
labels

['70482',
 '78700',
 '72197',
 '93306',
 '97546',
 'A9508',
 '0266U',
 '78020',
 '77263',
 '0271U',
 'Q0249',
 '0225U',
 '70491',
 '73562',
 'G0144',
 '76775',
 '0722T',
 '72147',
 'A9516',
 '74178',
 '76100',
 '76800',
 '77615',
 '55875',
 'G6001',
 '78469',
 '97110',
 'G0120',
 '78761',
 '73223',
 '31643',
 '97016',
 'G0451',
 '73100',
 '78740',
 '72193',
 '90732',
 '76810',
 '0045U',
 'A9558',
 'C9762',
 '70490',
 '71045',
 '95992',
 '93976',
 'P2029',
 '71100',
 '92612',
 '77262',
 '97162',
 '0052U',
 '93321',
 '70355',
 '93892',
 'P3001',
 '0059U',
 '74290',
 '70100',
 '76510',
 '97022',
 '75572',
 'G0124',
 '92507',
 '78428',
 '90756',
 'U0005',
 '73120',
 'A9607',
 '0287U',
 'G2251',
 '0032U',
 '0244U',
 '72159',
 '0246U',
 '0278U',
 'A9586',
 '77295',
 'G9143',
 '92133',
 '74170',
 'Q9983',
 '93356',
 'A9507',
 '76977',
 '70250',
 '74230',
 '70450',
 '86960',
 '72040',
 '93970',
 '77301',
 '78831',
 '76815',
 '77089',
 '72170',
 '93925',
 '76818',
 'A9532',
 '0307U',
 'G0141',


In [6]:
for key, value in enumerate(labels):
    print(value)

70482
78700
72197
93306
97546
A9508
0266U
78020
77263
0271U
Q0249
0225U
70491
73562
G0144
76775
0722T
72147
A9516
74178
76100
76800
77615
55875
G6001
78469
97110
G0120
78761
73223
31643
97016
G0451
73100
78740
72193
90732
76810
0045U
A9558
C9762
70490
71045
95992
93976
P2029
71100
92612
77262
97162
0052U
93321
70355
93892
P3001
0059U
74290
70100
76510
97022
75572
G0124
92507
78428
90756
U0005
73120
A9607
0287U
G2251
0032U
0244U
72159
0246U
0278U
A9586
77295
G9143
92133
74170
Q9983
93356
A9507
76977
70250
74230
70450
86960
72040
93970
77301
78831
76815
77089
72170
93925
76818
A9532
0307U
G0141
77417
0050U
0153U
77771
70130
A4642
G0328
61798
78799
A9546
0023U
A9505
91312
76511
97018
77370
73706
G0123
72084
0240U
0394T
76514
Q0240
A9700
73502
78122
71120
A9562
A9547
78015
0233U
78185
A9588
76885
97036
70548
77071
97602
72127
A9559
77074
76506
73552
76391
Q9959
90656
79403
A9602
73701
73060
A9517
78226
A9536
70498
73650
0235U
76377
A9529
93308
61770
78492
91301
0040U
0031U
A9587
G0103
0055

In [7]:
NUM_LABELS= len(labels)

id2label={id:label for id,label in enumerate(labels)}

label2id={label:id for id,label in enumerate(labels)}

In [8]:
label2id

{'70482': 0,
 '78700': 1,
 '72197': 2,
 '93306': 3,
 '97546': 4,
 'A9508': 5,
 '0266U': 6,
 '78020': 7,
 '77263': 8,
 '0271U': 9,
 'Q0249': 10,
 '0225U': 11,
 '70491': 12,
 '73562': 13,
 'G0144': 14,
 '76775': 15,
 '0722T': 16,
 '72147': 17,
 'A9516': 18,
 '74178': 19,
 '76100': 20,
 '76800': 21,
 '77615': 22,
 '55875': 23,
 'G6001': 24,
 '78469': 25,
 '97110': 26,
 'G0120': 27,
 '78761': 28,
 '73223': 29,
 '31643': 30,
 '97016': 31,
 'G0451': 32,
 '73100': 33,
 '78740': 34,
 '72193': 35,
 '90732': 36,
 '76810': 37,
 '0045U': 38,
 'A9558': 39,
 'C9762': 40,
 '70490': 41,
 '71045': 42,
 '95992': 43,
 '93976': 44,
 'P2029': 45,
 '71100': 46,
 '92612': 47,
 '77262': 48,
 '97162': 49,
 '0052U': 50,
 '93321': 51,
 '70355': 52,
 '93892': 53,
 'P3001': 54,
 '0059U': 55,
 '74290': 56,
 '70100': 57,
 '76510': 58,
 '97022': 59,
 '75572': 60,
 'G0124': 61,
 '92507': 62,
 '78428': 63,
 '90756': 64,
 'U0005': 65,
 '73120': 66,
 'A9607': 67,
 '0287U': 68,
 'G2251': 69,
 '0032U': 70,
 '0244U': 71,
 '

In [9]:
df_org.head()

,Code,Disease
328,70482,Ct orbit/ear/fossa w/ow/dye
689,78700,Kidney imaging morphol
413,72197,Mri pelvis w/o w/dye
720,93306,Tte w/doppler complete
244,97546,Work hardening add-on


In [10]:
df_org["labels"]=df_org.Code.map(lambda x: label2id[x.strip()])

In [11]:
df_org.head()

,Code,Disease,labels
328,70482,Ct orbit/ear/fossa w/ow/dye,0
689,78700,Kidney imaging morphol,1
413,72197,Mri pelvis w/o w/dye,2
720,93306,Tte w/doppler complete,3
244,97546,Work hardening add-on,4


In [12]:
#df_org.Code.value_counts().plot(kind='pie', figsize=(10,10))

In [13]:
#So we have single category for each disease

In [14]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-uncased", max_length=512)

In [15]:
#Using bert based uncased Multiclass identifier

In [16]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-uncased", num_labels=NUM_LABELS, id2label=id2label, label2id=label2id)
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [17]:
#Some model info "Bert base uncase multilingal"

In [18]:
SIZE = df_org.shape[0]

train_size = int(0.98 * SIZE)  # 98% for training
test_size = SIZE - train_size  # 2% for testing

train_texts = list(df_org.Disease[:train_size])
test_texts = list(df_org.Disease[train_size:])

train_labels = list(df_org.labels[:train_size])
test_labels = list(df_org.labels[train_size:])


In [19]:
len(train_texts)

1078

In [33]:
len(train_texts), len(val_texts), len(test_texts)


(550, 275, 276)

In [34]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings  = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [35]:
class DataLoader(Dataset):

    def __init__(self, encodings, labels):

        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):

        # Retrieve tokenized data for the given index
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Add the label for the given index to the item dictionary
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):

        return len(self.labels)

In [36]:
train_dataloader = DataLoader(train_encodings, train_labels)

val_dataloader = DataLoader(val_encodings, val_labels)

test_dataset = DataLoader(test_encodings, test_labels)

In [37]:
from transformers import TrainingArguments, Trainer

In [49]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):

    # Extract true labels from the input object
    labels = pred.label_ids
    
    # Obtain predicted class labels by finding the column index with the maximum probability
    preds = pred.predictions.argmax(-1)
    
    # Compute macro precision, recall, and F1 score using sklearn's precision_recall_fscore_support function
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    
    # Calculate the accuracy score using sklearn's accuracy_score function
    acc = accuracy_score(labels, preds)
    
    # Return the computed metrics as a dictionary
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }

In [69]:
training_args = TrainingArguments(
    output_dir='./BertBaseMultilingual',
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    num_train_epochs=100,
    per_device_train_batch_size=8,  # Increase batch size for efficiency
    per_device_eval_batch_size=32,
    warmup_steps=100,
    weight_decay=0.001,
    logging_strategy='steps',
    logging_dir='./multi-class-logs',
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,  # Increase evaluation frequency
    save_strategy="steps", # Enable mixed-precision training for efficiency
    load_best_model_at_end=True,
    # Adjust learning rate based on your experimentation
    learning_rate=2e-5,
)


In [70]:
trainer = Trainer(
    # the pre-trained model that will be fine-tuned 
    model=model,
     # training arguments that we defined above                        
    args=training_args,                 
    train_dataset=train_dataloader,         
    eval_dataset=val_dataloader,            
    compute_metrics= compute_metrics
)

In [71]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,0.655700,10.286288,0.014545,0.006369,0.005732,0.008493
100,0.636800,10.186468,0.014545,0.006424,0.005782,0.008565
150,0.608800,10.363202,0.014545,0.006452,0.005806,0.008602
200,0.623400,10.347384,0.014545,0.006356,0.005720,0.008475
250,0.561400,10.255168,0.014545,0.006424,0.005782,0.008565
300,0.563800,10.130994,0.014545,0.006356,0.005720,0.008475
350,0.557400,10.467677,0.014545,0.006508,0.005857,0.008677
400,0.511500,10.448176,0.014545,0.006466,0.005819,0.008621
450,0.499600,10.380476,0.014545,0.006397,0.005757,0.008529
500,0.441300,10.597912,0.014545,0.006410,0.005769,0.008547


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_cla

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_cla

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_cla

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_cla

...

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_cla

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_cla

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_cla

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_cla

TrainOutput(global_step=6900, training_loss=0.19088142505590466, metrics={'train_runtime': 25013.211, 'train_samples_per_second': 2.199, 'train_steps_per_second': 0.276, 'total_flos': 770538475080000.0, 'train_loss': 0.19088142505590466, 'epoch': 100.0})

In [1]:
q=[trainer.evaluate(eval_dataset=df_org) for df_org in [train_dataloader, val_dataloader, test_dataset]]

pd.DataFrame(q, index=["train","val","test"]).iloc[:,:5]

NameError: name 'train_dataloader' is not defined

# Testing 

In [37]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast,BertForSequenceClassification
df= pd.read_csv(ROOT_DIR + "diseaseOne.csv")

In [33]:
def predict(text):
   
    
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cuda")

   
    outputs = model(**inputs)

    probs = outputs[0].softmax(1)


    
    pred_label_idx = probs.argmax()


    pred_label = model.config.id2label[pred_label_idx.item()]

    return probs, pred_label_idx, pred_label

In [34]:
model_path = "BertMultiClass-classification-model"
#trainer.save_model(model_path)
#tokenizer.save_pretrained(model_path)

In [35]:
model_path = "BertMultiClass-classification-model"


model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer= BertTokenizerFast.from_pretrained(model_path)
nlp= pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Correct Predictions

In [19]:
nlp("Kidney imaging morphol")

[{'label': '78700', 'score': 0.7879894375801086}]

In [78]:
nlp("Ct pelvis w/dye")

[{'label': '72193', 'score': 0.7063923478126526}]

In [30]:
nlp("Iodine I-123 sod iodide mic")

[{'label': 'A9516', 'score': 0.8007793426513672}]

In [31]:
nlp("Ob us >/= 14 wks addl fetus")

[{'label': '76810', 'score': 0.7713539600372314}]

In [36]:
nlp("Unxpl cnst hrtbl do gn xprsn")

[{'label': '0266U', 'score': 0.8791385293006897}]

In [39]:
nlp("Split blood or products")

[{'label': '86985', 'score': 0.8397481441497803}]

In [41]:
nlp("Comt gene")

[{'label': '0032U', 'score': 0.7789485454559326}]

In [43]:
nlp("Onc aml dna detcj/nondetcj")

[{'label': '0023U', 'score': 0.8197428584098816}]

In [45]:
nlp("having trouble with this disease Tpmt nudt15 genes")

[{'label': '0034U', 'score': 0.6153490543365479}]

In [49]:
nlp("Cell enumeration phys id")   #  ----> Wrong prediction as starting 2 words seems similar that's why it cant pick

[{'label': '86152', 'score': 0.7760706543922424}]

In [38]:
df.head(40)

,Code,Disease
0,86152,Cell enumeration id
1,86153,Cell enumeration phys interp
2,86890,Autologous blood process
3,86891,Autologous blood op salvage
4,86927,Plasma fresh frozen
5,86930,Frozen blood prep
6,86931,Frozen blood thaw
7,86932,Frozen blood freeze/thaw
8,86945,Blood product/irradiation
9,86950,Leukacyte transfusion


# Observations / Change Needed

In [ ]:
#weight decay can be increased to 0.1 from 0.001 to resolve generalization

#augment my dataset like if multiple disease name letters match with each other then find the distinct names from these disease and augment the dataset with those distinct disease words against its diease code my dataset name is diseaseOne.csv

#Like if one disease code and disease name are like this : 86152	Cell enumeration id   and other other one is this: 86153	Cell enumeration phys interp then place "phys interp" with 86153 as a new row in dataset 


# Data Augmentation

In [ ]:
# Adding distinct names of disease in a new row against it code
# Chaning diseae name word sequance and adding them against their disease code

In [80]:
import pandas as pd

# Load your original dataset
df = pd.read_csv('diseaseOne.csv')

# Initialize an empty list to store DataFrames
dfs = []

# Iterate through each row in the original dataset
for index, row in df.iterrows():
    code = row['Code']
    disease = row['Disease']

    # Split the disease name into words
    words = disease.split()

    # Create a DataFrame for the original row
    data = {'Code': [code], 'Disease': [disease]}
    original_df = pd.DataFrame(data)

    # Append the original DataFrame to the list
    dfs.append(original_df)

    # Create new rows for different word sequences (bigrams)
    for i in range(len(words) - 1):
        bigram = ' '.join(words[i:i+2])
        if bigram != disease:  # Exclude the original sequence
            data = {'Code': [code], 'Disease': [bigram]}
            df_part = pd.DataFrame(data)

            # Append the new DataFrame to the list
            dfs.append(df_part)

    # Shuffle the order of words in the disease name
    shuffled_words = random.sample(words, len(words))
    shuffled_sequence = ' '.join(shuffled_words)

    if shuffled_sequence != disease:  # Exclude the original sequence
        data = {'Code': [code], 'Disease': [shuffled_sequence]}
        df_part = pd.DataFrame(data)

        # Append the new DataFrame to the list
        dfs.append(df_part)

# Concatenate all DataFrames in the list into one
augmented_df = pd.concat(dfs, ignore_index=True)

# Save the augmented dataset to a new CSV file
augmented_df.to_csv('augmented_disease.csv', index=False)


In [81]:
augmented_df.shape

(5523, 2)

In [54]:
df.shape

(1101, 2)

In [82]:
augmented_df.head(60)

#Here 2 Augments

,Code,Disease
0,86152,Cell enumeration id
1,86152,Cell enumeration
2,86152,enumeration id
3,86152,enumeration id Cell
4,86153,Cell enumeration phys interp
5,86153,Cell enumeration
6,86153,enumeration phys
7,86153,phys interp
8,86153,phys interp Cell enumeration
9,86890,Autologous blood process
